In [1]:
import tensorflow as tf
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

def get_y(x):
    return 10 + x*x

def get_z(y):
    return 10 + y

def sample_data(n=10000, scale=100):
    data = []

    x = scale*(np.random.random_sample((n,))-0.5)

    for i in range(n):
        yi = get_y(x[i])
        zi = get_z(yi)
        data.append([x[i], yi,zi])

    return np.array(data)


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def generator(Z,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Generator",reuse=reuse):
        h1 = tf.layers.dense(Z,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        out = tf.layers.dense(h2,3)

    return out

def discriminator(X,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Discriminator",reuse=reuse):
        h1 = tf.layers.dense(X,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        h3 = tf.layers.dense(h2,3)
        out = tf.layers.dense(h3,1)

    return out, h3

with tf.device('/gpu:0'):
  X = tf.placeholder(tf.float32,[None,3])
  Z = tf.placeholder(tf.float32,[None,3])

  G_sample = generator(Z)
  r_logits, r_rep = discriminator(X)
  f_logits, g_rep = discriminator(G_sample,reuse=True)

  disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,labels=tf.ones_like(r_logits)) + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.zeros_like(f_logits)))
  gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.ones_like(f_logits)))

  gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
  disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Discriminator")

  gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss,var_list = gen_vars) # G Train step
  disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_loss,var_list = disc_vars) # D Train step



  # sess = tf.Session(config=config)
  sess = tf.Session()
  tf.global_variables_initializer().run(session=sess)

  batch_size = 256
  nd_steps = 10
  ng_steps = 10

  x_plot = sample_data(n=batch_size)

  for i in range(5):
      X_batch = sample_data(n=batch_size)
      Z_batch = sample_Z(batch_size, 3)

      for _ in range(nd_steps):
          _, dloss = sess.run([disc_step, disc_loss], feed_dict={X: X_batch, Z: Z_batch})
      rrep_dstep, grep_dstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

      for _ in range(ng_steps):
          _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

      rrep_gstep, grep_gstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

      print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss))

    # if i%10000 == 0:
    #     # plt.figure()
    #     g_plot = sess.run(G_sample, feed_dict={Z: Z_batch})
        # from mpl_toolkits import mplot3d

        # ax = plt.axes(projection = '3d')
        # xax = ax.scatter3D(x_plot[:,0],x_plot[:,1],x_plot[:,2])
        # # xax = plt.scatter(x_plot[:,0], x_plot[:,1])
        # gax = ax.scatter3D(g_plot[:,0],g_plot[:,1],g_plot[:,2])
        # # gax = plt.scatter(g_plot[:,0],g_plot[:,1])

        # plt.legend((xax,gax), ("Real Data","Generated Data"))
        # plt.title('Samples at Iteration %d'%i)
        # plt.tight_layout()
        # plt.savefig('pic.png')
        # plt.close()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Iterations: 0	 Discriminator loss: 0.6749	 Generator loss: 0.7143
Iterations: 1	 Discriminator loss: 0.6547	 Generator loss: 0.7343
Iterations: 2	 Discriminator loss: 0.6281	 Generator loss: 0.7628
Iterations: 3	 Discriminator loss: 0.6007	 Generator loss: 0.7936
Iterations: 4	 Discriminator loss: 0.5705	 Generator loss: 0.8273


In [0]:
gen_vars

[<tf.Variable 'GAN/Generator/dense/kernel:0' shape=(3, 16) dtype=float32_ref>,
 <tf.Variable 'GAN/Generator/dense/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'GAN/Generator/dense_1/kernel:0' shape=(16, 16) dtype=float32_ref>,
 <tf.Variable 'GAN/Generator/dense_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'GAN/Generator/dense_2/kernel:0' shape=(16, 3) dtype=float32_ref>,
 <tf.Variable 'GAN/Generator/dense_2/bias:0' shape=(3,) dtype=float32_ref>]

In [0]:
# #3d plotting 
# from mpl_toolkits import mplot3d
# ax = plt.axes(projection='3d')
# ax.scatter3D()

In [0]:
import tensorflow as tf


device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
  raise SystemError('GPU NOT FOUND')
else:
  print("Found GPU at : {}".format(device))

Found GPU at : /device:GPU:0


In [0]:
X_batch.shape

(256, 3)

In [0]:
Z_batch.shape

(256, 3)